## LCEL 인터페이스

사용자 정의 체인을 가능한 쉽게 만들 수 있도록, [`Runnable`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable) 프로토콜을 구현했습니다. 

`Runnable` 프로토콜은 대부분의 컴포넌트에 구현되어 있습니다.

이는 표준 인터페이스로, 사용자 정의 체인을 정의하고 표준 방식으로 호출하는 것을 쉽게 만듭니다.
표준 인터페이스에는 다음이 포함됩니다.

- [`stream`](#stream): 응답의 청크를 스트리밍합니다.
- [`invoke`](#invoke): 입력에 대해 체인을 호출합니다.
- [`batch`](#batch): 입력 목록에 대해 체인을 호출합니다.

비동기 메소드도 있습니다.

- [`astream`](#async-stream): 비동기적으로 응답의 청크를 스트리밍합니다.
- [`ainvoke`](#async-invoke): 비동기적으로 입력에 대해 체인을 호출합니다.
- [`abatch`](#async-batch): 비동기적으로 입력 목록에 대해 체인을 호출합니다.
- [`astream_log`](#async-stream-intermediate-steps): 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다.

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI()
prompt = PromptTemplate(template="{topic}에 대하여 3문장으로 설명해줘")
chain = prompt | model | StrOutputParser()

## stream : 실시간 출력

In [4]:
for token in chain.stream({"topic": "멀티모달"}):
    print(token, end="", flush=True)

멀티모달은 여러 가지 방식으로 정보를 전달하는 시스템을 의미합니다. 주로 음성, 이미지, 텍스트, 동영상 등 다양한 매체를 활용하여 사용자에게 정보를 제공하며, 사용자가 선호하는 방식으로 소통할 수 있습니다. 이를 통해 더 효과적이고 효율적인 정보 전달이 가능해지며, 사용자 경험을 향상시킬 수 있습니다.

## invoke:호출

In [5]:
chain.invoke({"topic":"ChatGPT"})

'ChatGPT는 인공 지능 챗봇으로, 자연어 처리 기술을 사용하여 대화 상대와 의사소통한다. 사용자의 질문이나 요청에 즉각적으로 응답하고 다양한 주제에 대해 대화할 수 있다. ChatGPT는 학습과 발전을 거듭해 보다 자연스러운 대화를 제공하는 데 주력하고 있습니다.'

## batch: 배치(단위 실행)

In [6]:
chain.batch([{"topic":"ChatGPT"}, {"topic":"Instagram"}])

['ChatGPT는 인공지능 기술을 이용하여 자연스러운 대화를 제공하는 채팅 로봇입니다. 사용자의 질문에 대답하고 이야기를 주도하여 정보를 제공하며, 다양한 주제에 대해 대화할 수 있습니다. ChatGPT는 사람과의 상호작용을 통해 학습하고 성장하며, 사용자의 요구에 맞는 맞춤형 서비스를 제공합니다.',
 'Instagram은 사진과 동영상을 공유할 수 있는 소셜 미디어 플랫폼으로, 사용자들은 자신의 일상을 시각적으로 공유하고 서로 소통할 수 있습니다. 해시태그를 활용하여 관심사나 주제별로 검색하고 팔로우할 수 있으며, 인기 있는 콘텐츠를 탐색할 수도 있습니다. 또한 스토리 기능을 통해 실시간으로 일상을 공유하고 소통할 수 있습니다.']

In [7]:
chain.batch(
    [
        {"topic":"ChatGPT"},
        {"topic":"Instagram"},
        {"topic":"멀티모달"},
        {"topic":"프로그래밍"},
        {"topic":"머신러닝"}
    ],
    config={"max_concurrency": 3},
)

['ChatGPT는 온라인 상에서 자연어 처리와 대화를 제공하는 인공지능 기술이다. 사용자와의 상호작용을 통해 학습하며 지속적으로 발전해나가는 챗봇 시스템이다. 다양한 주제에 대해 대화를 나누고 질문에 답변하는 데에 활용된다.',
 'Instagram은 사진과 동영상을 공유하는 소셜 미디어 플랫폼으로, 사용자들은 자신의 일상을 쉽게 공유하고 소통할 수 있다. 피드를 통해 팔로워들의 소식을 접하고 좋아요나 댓글로 소통할 수 있으며, 인기 있는 해시태그를 활용하여 다양한 주제의 콘텐츠를 찾아볼 수 있다. 또한 스토리 기능을 활용하여 24시간 동안 임시적으로 컨텐츠를 공유할 수 있어 더 자유롭고 빠르게 소통할 수 있다.',
 '멀티모달은 여러 가지 형태의 미디어(예: 텍스트, 이미지, 음성, 비디오)를 함께 사용하여 정보를 전달하는 방식을 말합니다. 이를 통해 사용자들은 다양한 감각을 동시에 활용하여 보다 풍부한 경험을 얻을 수 있습니다. 멀티모달은 정보의 이해와 기억을 증진시키며, 사용자들에게 보다 효과적인 커뮤니케이션을 제공합니다.',
 '프로그래밍은 컴퓨터에게 명령을 전달해 원하는 작업을 수행하도록 하는 과정이다. 이를 위해 프로그래머는 프로그래밍 언어를 사용하여 알고리즘을 작성하고 코드를 작성해야 한다. 코드를 작성하고 실행함으로써 프로그래머는 문제를 해결하고 원하는 결과를 얻을 수 있다.',
 '머신러닝은 컴퓨터 시스템이 데이터를 분석하고 학습하여 패턴을 파악하는 인공지능 기술이다. 이를 통해 예측, 분류, 군집 등의 작업을 수행하며, 대표적인 알고리즘으로는 신경망, 의사결정 트리, SVM 등이 있다. 머신러닝은 다양한 산업 분야에서 활발히 활용되며, 데이터 기반의 의사결정과 자동화된 프로세스 개선에 기여하고 있다.']

## async stream : 비동기 스트림

In [8]:
async for token in chain.astream({"topic": "YouTube"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(token, end="", flush=True)

YouTube는 동영상을 감상하고 업로드할 수 있는 온라인 동영상 공유 플랫폼이다. 사용자는 다양한 콘텐츠를 시청하며 즐길 수 있고, 자신의 콘텐츠를 업로드하여 다른 사람들과 공유할 수 있다. 또한, 광고 수익을 통해 크리에이터들이 수익을 창출할 수 있는 플랫폼이기도 하다.

## async invoke: 비동기 호출

In [9]:
my_process = chain.ainvoke({"topic":"NVDA"})

In [10]:
await my_process

'NVDA는 미국의 컴퓨터 소프트웨어 기업인 Nvidia Corporation의 주식을 나타내는 주식 코드이다. 주로 그래픽 처리 장치와 관련된 제품을 개발하며 시장에서 선도적인 위치를 차지하고 있다. 기술 산업에서 성장 가능성이 높은 기업 중 하나로 평가받고 있다.'

## async batch : 비동기 배치

In [11]:
my_abatch_process = chain.abatch(
    [{"topic":"YouTube"}, {"topic":"Instagram"}, {"topic": "Facebook"}]
)

In [12]:
await my_abatch_process

['YouTube는 사용자들이 동영상을 업로드하고 시청할 수 있는 온라인 비디오 플랫폼이다. \n세계적으로 가장 인기 있는 온라인 동영상 서비스 중 하나로, 다양한 콘텐츠를 제공하며 크리에이터와 시청자들을 연결시켜준다.\n유익한 정보, 엔터테인먼트, 교육 등 다양한 주제의 동영상을 제공하여 사용자들에게 다양한 즐거움을 주고 있다.',
 '인스타그램은 사진과 동영상을 공유하고 소셜 네트워크 기능을 제공하는 SNS 플랫폼입니다. 사용자들은 팔로워들과 소통하며 일상을 공유하고 다양한 콘텐츠를 감상할 수 있습니다. 해시태그를 통해 관심사에 맞는 게시물을 찾을 수 있습니다.',
 'Facebook은 전세계적으로 가장 인기 있는 소셜 네트워킹 서비스로 사용자들이 친구들과 소통하고 콘텐츠를 공유하는 플랫폼이다. 사용자는 자신의 프로필을 통해 개인 정보를 공개하고 다양한 커뮤니티에 참여할 수 있다. 또한 광고와 비즈니스 영역에서도 활발하게 활용되는 플랫폼이다.']

## Parallel: 병렬성

LangChain Expression Language가 병렬 요청을 지원하는 방법을 살펴봅시다.
예를 들어, `RunnableParallel`을 사용할 때, 각 요소를 병렬로 실행합니다.


In [13]:
from langchain_core.runnables import RunnableParallel

chain1 = (
    PromptTemplate.from_template("{country}의 수도는 어디야?")
    | model
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country}의 면적은 얼마야?")
    | model
    |StrOutputParser()
)

combined = RunnableParallel(capital=chain1, area=chain2)

In [15]:
chain1.invoke({"country":"대한민국"})

'대한민국의 수도는 서울이야.'

In [ ]:

chain2.invoke({"country":"미국"})

'미국의 면적은 대략 약 9,800만 제곱 킬로미터 (3,800만 제곱 마일) 입니다.'

In [16]:
combined.invoke({"country":"대한민국"})

{'capital': '대한민국의 수도는 서울이다.', 'area': '대한민국의 총 면적은 약 100,363 km² 입니다.'}

In [17]:
chain1.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 수도는 서울입니다.', '미국의 수도는 워싱턴 D.C.입니다.']

In [18]:
chain2.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 면적은 100,363.39 km² 입니다.',
 '미국의 총 면적은 대략 9,826만 제곱킬로미터(약 3,796만 제곱마일)입니다.']

In [19]:
combined.batch([{"country": "대한민국"}, {"country": "미국"}])

[{'capital': '대한민국의 수도는 서울입니다.', 'area': '대한민국의 면적은 약 100,363.77㎢ 입니다.'},
 {'capital': '미국의 수도는 워싱턴 D.C.입니다.', 'area': '미국의 총 면적은 대략 9,833만 제곱킬로미터입니다.'}]